<a href="https://colab.research.google.com/github/SilverSurferClash/Sales_analysis/blob/main/Import_clean_SFDC_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
#!pip freeze > requirements.txt

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import seaborn as sns
import os
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.set_option('display.precision', 2)
#Use option to round floats to two decimals
#pd.set_option('display.float_format',  '{:,.2f}'.format)
# Use option to display $ dollars
#pd.set_option('display.float_format', '${:.2f}'.format)

# Plotting pretty figures and avoid blurry images
#%config InlineBackend.figure_format = 'retina'
# Larger scale for plots in notebooks
#sns.set_context('notebook')


# Enable multiple cell outputs
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = 'all'

import warnings
warnings.filterwarnings('ignore')              


In [25]:
os.getcwd()

'/content/drive/My Drive/Colab_Notebooks/Sales_Analysis'

In [26]:
os.chdir("/content/drive/MyDrive/Colab_Notebooks/Sales_Analysis")

In [27]:
import_sfdc = pd.read_excel("./All Data EUAA-2022-10-25-12-29-00.xlsx", sheet_name = 0, 
                            skiprows = 10)
import_sfdc = import_sfdc.drop(["Unnamed: 0", "Unnamed: 2"], axis = 1)
import_sfdc.head()

,Accession #,Reporting Country,Billing Type,LIMS Project ID,Account: Account Name,Contact: Full Name,Date Received,Final Report Date,Test Type,Cancer Type
0,A0214386,CH,Pharma : ROW,AMG_01,AMG_01: Universitatsspital Zurich,Alessandra Curioni Fontecedro,11/25/2019,NaN,Guardant 360,Lung adenocarcinoma
1,A0214865,CA,Pharma : ROW,AMG_01,AMG_01: Princess Margaret Cancer Centre,Adrian Sacher,11/26/2019,NaN,Guardant 360,Lung adenocarcinoma
2,A0215878,BE,Pharma : ROW,AMG_01,AMG_01: Universitair Ziekenhuis Antwerpen,Hans Prenen,11/29/2019,NaN,Guardant 360,Lung adenocarcinoma
3,A0215958,DE,Pharma : US,AMG_01,AMG_01: Universitatsklinikum Koln,Wolf Juergen,11/30/2019,NaN,Guardant 360,Lung adenocarcinoma
4,A0215959,DE,Pharma : ROW,AMG_01,AMG_01: Universitatsklinikum Koln,Wolf Juergen,11/30/2019,NaN,Guardant 360,Lung adenocarcinoma


In [28]:
import_sales = pd.read_excel("./Sales_Volume_EUAA_Analytics.xlsx", sheet_name = 0, skiprows = 28)
import_sales.head(2)

,2019-09-01 00:00:00,Account: Reporting Country ↑,Date Received,Accession #,Account: Account Name,Contact: Full Name,Billing Type,Final Report Date,Invoiced Ammount,Expected Amount $,Invoiced/In Prg.,FOC,Collected,Unnamed: 13,Unnamed: 14,Unnamed: 15,Total,1943954.40925
0,NaN,BR,9/30/2019,A0198807,Rede D'Or Hospital,Maria Ignez Braghiroli,Self-Pay Patient,10/6/2019,0,0,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN
1,1 EUR = 1.1048 USD,NaN,9/18/2019,A0196311,Rede D'Or Hospital,Milena Mak,Self-Pay Patient,9/26/2019,0,0,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN


#### Cleaning up the spreadsheet with the revenue information

In [29]:
# Select only rows that start with A0
#mask_remove sum columns - (i) remove NA ib Accession columns (ii) remove rows with no "A0"
#Remove first columns
#Convert received and final report date to datetime objects

In [31]:
mask_isna = import_sales['Accession #'].isna()

In [32]:
import_sales = import_sales[~mask_isna]

In [35]:
mask_startswith = import_sales['Accession #'].str.startswith("A0")

In [39]:
import_sales = import_sales[mask_startswith]
import_sales = import_sales.drop([import_sales.columns[0]], axis = 1)

In [48]:
import_sales["Date Received"] = pd.to_datetime(import_sales["Date Received"])

In [51]:
import_sales["Expected Amount $"]

0                                           0
1                                           0
3                                        3490
4                                        3490
6                                        3490
                        ...                  
4173    Accounts Receivable  - G360 - 83 GENE
4174                     Accounts Receivable 
4175                     Accounts Receivable 
4176             G360 Experience Project 7/15
4177    Accounts Receivable  - G360 - 83 GENE
Name: Expected Amount $, Length: 3660, dtype: object

In [49]:
import_sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3660 entries, 0 to 4177
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Account: Reporting Country  ↑  404 non-null    object        
 1   Date Received                  3660 non-null   datetime64[ns]
 2   Accession #                    3660 non-null   object        
 3   Account: Account Name          3660 non-null   object        
 4   Contact: Full Name             3659 non-null   object        
 5   Billing Type                   3660 non-null   object        
 6   Final Report Date              3424 non-null   object        
 7   Invoiced Ammount               3639 non-null   object        
 8   Expected Amount $              2467 non-null   object        
 9   Invoiced/In Prg.               361 non-null    object        
 10  FOC                            716 non-null    object        
 11  Collected        

In [57]:
import_sales.shape, import_sfdc.shape

((3660, 17), (14982, 10))

In [33]:
import_sales.merge()